In [1]:
import sqlite3
import random
import re

In [2]:
def create_question(start, end, count):
    conn = sqlite3.connect('Quran')
    cursor = conn.cursor()

    cursor.execute('SELECT Id, Aye FROM AyeMatn WHERE Id BETWEEN ? AND ?', (start, end))
    verses = cursor.fetchall()

    if not verses:
        return "هیچ آیهای در این محدوده یافت نشد."

    all_words = []
    for verse in verses:
        all_words.extend(verse[1].split())

    unique_words = list(set(all_words))
    questions = []

    def find_similar_words(word):
        pattern = rf"\b{word[:2]}\w*\b"
        return [w for w in unique_words if re.match(pattern, w) and w != word]

    for _ in range(count):
        verse = random.choice(verses)
        verse_id, aye_text = verse
        words = aye_text.split()

        valid_words = [word for word in words if len(word) > 2]
        if not valid_words:
            continue

        correct_word = random.choice(valid_words)
        blank_index = words.index(correct_word)

        distractors = find_similar_words(correct_word)

        if len(distractors) < 3:
            additional_candidates = [
                w for w in unique_words if w != correct_word and abs(len(w) - len(correct_word)) <= 1
            ]
            distractors += random.sample(additional_candidates, min(3 - len(distractors), len(additional_candidates)))

        options = distractors[:3] + [correct_word]
        random.shuffle(options)

        question_text = " ".join(["______" if i == blank_index else word for i, word in enumerate(words)])

        questions.append({
            "verse_id": verse_id,
            "question": question_text,
            "options": options,
            "answer": correct_word
        })

    conn.close()
    return questions

In [3]:
questions = create_question(10, 20, 5)
for i, q in enumerate(questions, 1):
    print(f"سوال {i} (آیه {q['verse_id']}):")
    print(q['question'])
    print("گزینه‌ها:")
    for j, opt in enumerate(q['options'], 1):
        print(f"{j}. {opt}")
    print(f"پاسخ صحیح: {q['answer']}\n")

سوال 1 (آیه 13):
إِنَّ الَّذِينَ كَفَرُوا سَوَاءٌ عَلَيْهِمْ أَأَنذَرْتَهُمْ أَمْ ______ تُنذِرْهُمْ لَا يُؤْمِنُونَ
گزینه‌ها:
1. لَهُمْ
2. لَمْ
3. لَّا
4. لَا
پاسخ صحیح: لَمْ

سوال 2 (آیه 10):
الَّذِينَ ______ بِالْغَيْبِ وَيُقِيمُونَ الصَّلَاةَ وَمِمَّا رَزَقْنَاهُمْ يُنفِقُونَ
گزینه‌ها:
1. يُوقِنُونَ
2. يُنفِقُونَ
3. يُؤْمِنُونَ
4. يُخَادِعُونَ
پاسخ صحیح: يُؤْمِنُونَ

سوال 3 (آیه 12):
أُولَـٰئِكَ عَلَىٰ ______ مِّن رَّبِّهِمْ ۖ وَأُولَـٰئِكَ هُمُ الْمُفْلِحُونَ
گزینه‌ها:
1. هُمُ
2. هُدًى
3. هُمْ
4. هُم
پاسخ صحیح: هُدًى

سوال 4 (آیه 18):
وَإِذَا قِيلَ لَهُمْ لَا تُفْسِدُوا فِي ______ قَالُوا إِنَّمَا نَحْنُ مُصْلِحُونَ
گزینه‌ها:
1. الْآخِرِ
2. النَّاسِ
3. النَّاسُ
4. الْأَرْضِ
پاسخ صحیح: الْأَرْضِ

سوال 5 (آیه 16):
يُخَادِعُونَ اللَّهَ وَالَّذِينَ آمَنُوا ______ يَخْدَعُونَ إِلَّا أَنفُسَهُمْ وَمَا يَشْعُرُونَ
گزینه‌ها:
1. وَمَا
2. وَلَـٰكِن
3. وَعَلَىٰ
4. وَلَهُمْ
پاسخ صحیح: وَمَا

